# Collecting and Cleaning SPY 1-Minute Data

This notebook is the **first step** of our project.  
Here, we prepare a **clean and consistent** SPY 1-minute dataset that all later notebooks can safely use.

We do four main things:

1. **Load raw SPY data**

   - We read the original SPY 1-minute OHLCV data from:  
     **`data/raw/`**
   - This raw file contains:
     - **Time** (as Unix timestamps)
     - **Open, High, Low, Close**
     - **Volume**

2. **Convert time to U.S. Eastern Time**

   - The raw timestamps are in **Unix time** (seconds since 1970-01-01 UTC).
   - We convert these timestamps into **human-readable datetimes** in  
     **U.S. Eastern Time (ET)**.
   - This makes it much easier to:
     - Work with **regular session hours** (09:30–16:00 ET)
     - Align with typical trading concepts like **Initial Balance (09:30–10:30)**.

3. **Simplify per-minute price fields**

   - For each 1-minute bar, we **drop the `open` column**.
   - We **keep** only:
     - **high**
     - **low**
     - **close**
     - **volume**
   - This reduces noise and keeps the dataset focused on the variables we actually use later.

4. **Save the cleaned dataset**

   - After cleaning and simplifying, we save the result as a new file in:  
     **`data/clean/`**
   - This cleaned file will be the **standard input** for all later notebooks in the project.
   - Any later analysis (IB, AVWAP, labels, etc.) will start from this **clean, consistent SPY 1-minute dataset**.


In [52]:
from pathlib import Path
import pandas as pd

In [53]:
#looking at the data to see if its in correct folders

PROJECT_ROOT = Path("..").resolve()

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN = PROJECT_ROOT / "data" / "clean"

RAW_FILE = DATA_RAW / "spy_1min_bats_2025.csv"

print("RAW_FILE path:", RAW_FILE)
print("File exists?:", RAW_FILE.exists())


RAW_FILE path: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/raw/spy_1min_bats_2025.csv
File exists?: True


## 1) Reading raw SPY 1-minute OHLCV + volume data

We start by **loading the raw SPY 1-minute data** so we can see exactly what we are working with.

Before we clean or transform anything, we want to:

- Inspect the **structure** of the file
- Check the **column names**
- Confirm that the data matches what we expect

This is also the moment where we can do basic **exploratory data analysis (EDA)** on the raw data  
(e.g., look at the first few rows, check types, look for obvious issues).


### Expected columns

Our raw file should contain exactly **six columns**, with the following names:

- **time** → Unix timestamp (seconds since 1970-01-01)
- **open** → opening price of the 1-minute bar
- **high** → highest price within that minute
- **low** → lowest price within that minute
- **close** → closing price of the 1-minute bar
- **Volume** → traded volume in that minute

If the data loads correctly and we see these six columns,  
then we know we can safely move on to the **cleaning and transformation** steps.


In [54]:
#checking raw data head

df_raw = pd.read_csv(RAW_FILE)
df_raw.head()

,time,open,high,low,close,Volume
0,1757338200,648.63,648.86,648.24,648.260,141588
1,1757338260,648.26,648.45,648.15,648.270,42118
2,1757338320,648.30,648.46,648.10,648.260,37143
3,1757338380,648.28,648.47,648.23,648.400,42231
4,1757338440,648.40,648.68,648.32,648.665,23659


In [55]:
#checking column dtypes and their statistics so that can easily seen missing or incorrect valuation

df_raw.info()
df_raw.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21450 entries, 0 to 21449
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    21450 non-null  int64  
 1   open    21450 non-null  float64
 2   high    21450 non-null  float64
 3   low     21450 non-null  float64
 4   close   21450 non-null  float64
 5   Volume  21450 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 1005.6 KB


,time,open,high,low,close,Volume
count,2.145000e+04,21450.000000,21450.000000,21450.000000,21450.000000,2.145000e+04
mean,1.760548e+09,668.012768,668.153902,667.867178,668.010579,3.136366e+04
std,1.917741e+06,9.477913,9.467176,9.486371,9.477293,3.576077e+04
min,1.757338e+09,647.330000,647.510000,647.220000,647.310000,1.314000e+03
25%,1.758825e+09,661.130000,661.330000,660.912250,661.122500,1.461550e+04
50%,1.760547e+09,666.830000,666.970000,666.660000,666.820000,2.288800e+04
75%,1.762272e+09,673.017500,673.120000,672.870000,673.000000,3.698300e+04
max,1.763759e+09,689.600000,689.700000,689.520000,689.590000,1.362579e+06


## 2) Converting Unix Timestamps to Real New York Dates and Times

Our raw time column is in **Unix Timestamp** format.

- A **Unix Timestamp** is just:
  > The number of seconds that have passed since **1970-01-01 00:00:00 (UTC)**.
- This is great for computers, but **not easy for humans** to read or reason about.

Because we are working with **U.S. stock market data**, we care about the time in:

- **U.S. Eastern Time (New York time)**

This is important because:

- The regular trading session is defined in **New York time**  
  (for example, **09:30–16:00 ET**, and **09:30–10:30 ET** for the Initial Balance).
- When we say “this bar is at 09:45”, it should mean **09:45 in New York**, not in UTC.



### What we do in this step

1. Take the **`time`** column (Unix timestamps).
2. Convert each value into a **timezone-aware datetime** in:
   - **U.S. Eastern Time (ET)**
3. Store the result in a clear datetime column (for example: **`datetime`**).

After this conversion, each row will have:

- A **human-readable date** (year-month-day)
- A **clear clock time** (hour:minute:second)
- All correctly aligned to **New York time**

This makes it much easier to:

- Filter by session hours (e.g., **09:30–16:00**)
- Extract the **Initial Balance window (09:30–10:30)**
- Do any intraday analysis in a way that matches **real trading hours**.

In [56]:
import numpy as np

# Our data includes both DST (Daylight Saving Time) and ST (Standart Time) in U.S. Eastern Time.
# Because Americans setted their clocks back 1 hour in November 2nd at 02:00 (in DST) and 01:00 (in ST)
# After November 2nd, U.S. Eastern Time is in ST timeframe now which is 1 hour setted back from DST timeframe

# Unix timestamp 1762063200 --> November 2nd at 02:00 (in DST) and 01:00 (in ST)
threshold = 1762063200

# created new column 'datetime' which considers times before November 2nd at 02:00 as DST and after November 2nd at 01:00 as ST and converts to New York Timeframe
df_raw["datetime"] = np.where(
    df_raw["time"] < threshold,
    pd.to_datetime(df_raw["time"], unit="s") - pd.Timedelta(hours=4),
    pd.to_datetime(df_raw["time"], unit="s") - pd.Timedelta(hours=5)
)

df_raw.drop(columns=["time"], inplace=True)

df_raw.head()

,open,high,low,close,Volume,datetime
0,648.63,648.86,648.24,648.260,141588,2025-09-08 09:30:00
1,648.26,648.45,648.15,648.270,42118,2025-09-08 09:31:00
2,648.30,648.46,648.10,648.260,37143,2025-09-08 09:32:00
3,648.28,648.47,648.23,648.400,42231,2025-09-08 09:33:00
4,648.40,648.68,648.32,648.665,23659,2025-09-08 09:34:00


In [ ]:
# I adjusting our 'datetime' column as our first column

cols = df_raw.columns.tolist()
cols.remove('datetime')     
new_order = ['datetime'] + cols
df_raw = df_raw[new_order]

df_raw.head()

,datetime,open,high,low,close,Volume
0,2025-09-08 09:30:00,648.63,648.86,648.24,648.260,141588
1,2025-09-08 09:31:00,648.26,648.45,648.15,648.270,42118
2,2025-09-08 09:32:00,648.30,648.46,648.10,648.260,37143
3,2025-09-08 09:33:00,648.28,648.47,648.23,648.400,42231
4,2025-09-08 09:34:00,648.40,648.68,648.32,648.665,23659


In [63]:
# We need to see our data's first 1 min-candle and our data's last minute candle to see if data catches correct U.S. datetime interval
# Which is in our data with getting 55 days from 8th September to 21st November with correct stock market hours 09:00 - 16:00

# Moreover, we can see it from whole data summary with .describe() function

df_raw['datetime'].info()
df_raw['datetime'].describe()


<class 'pandas.core.series.Series'>
RangeIndex: 21450 entries, 0 to 21449
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
21450 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 167.7 KB


count                  21450
mean     2025-10-15 12:44:30
min      2025-09-08 09:30:00
25%      2025-09-25 14:22:15
50%      2025-10-15 12:44:30
75%      2025-11-04 11:06:45
max      2025-11-21 15:59:00
Name: datetime, dtype: object

## 3) Removing the per-minute `open` and keeping only `high`, `low`, `close`, and `volume`

In our project, we **do not need** the **per-minute `open` price**.

Instead, we focus on:

- **high**
- **low**
- **close**
- **volume**

There are two main reasons for dropping the `open` column:

1. **Simplicity for labels and logic**

   - For 1-minute bars, the **close price** is a very common reference.
   - Using only the **close** makes it easier to:
     - Define labels (e.g., “did the price go up or down after this minute?”)
     - Build and read our rules without extra noise.

2. **Less clutter in the dataset**

   - Every extra column adds complexity.
   - Since we are **not using** the `open` in our hypotheses or features,  
     keeping it would just make the dataframe heavier and more confusing.


### What we do in this step

- We **drop the `open` column** from our dataframe.
- We **keep only**:

  - **high**
  - **low**
  - **close**
  - **volume**

After this step, each 1-minute row includes only the price and volume information we actually plan to use in the rest of the project.


In [64]:
df_raw.drop(columns=["open"], inplace=True)
df_raw.head()

,datetime,high,low,close,Volume
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659


## 4) Saving the cleaned dataset into `data/clean/` for later use

At this point, we have:

- Loaded the **raw SPY 1-minute data**
- Converted **Unix timestamps** into **New York (U.S. Eastern) time**
- Removed the unnecessary **`open`** column
- Kept only the fields we need:
  - **time / datetime**
  - **high**
  - **low**
  - **close**
  - **volume**

Now this dataframe is our **clean, standard version** of the SPY 1-minute data.

### Why do we save it in `data/clean/`?

We want a **clear separation** between:

- **Raw data** (as downloaded) → stored in **`data/raw/`**
- **Cleaned data** (ready for analysis) → stored in **`data/clean/`**

By saving the cleaned dataframe into **`data/clean/`**, we:

- Make it easy for **all later notebooks** to load the same consistent dataset.
- Avoid repeating the same cleaning steps over and over.
- Create a clear “pipeline”:
  - **raw → clean → features → models**

### What we do in this step

- Take the final cleaned dataframe.
- Save it as a file (for example, a `.csv`) into:

> **`data/clean/`**

From now on, whenever another notebook needs SPY 1-minute data,  
it will **directly use this cleaned file** instead of the raw one.


In [65]:
from pathlib import Path

# 1) Define project root which is the main branch in our repository
PROJECT_ROOT = Path("..").resolve()

# 2) We need to go to data/clean folder so define that pathway
DATA_CLEAN = PROJECT_ROOT / "data" / "clean"
DATA_CLEAN.mkdir(parents=True, exist_ok=True)  # yoksa oluştur

# 3) Kaydedilecek CSV dosyasının adı
clean_csv_path = DATA_CLEAN / "spy_1min_et_clean.csv"

# 4) df_raw şu anda en son işlenmiş (temiz) halinse, onu kaydediyoruz
df_raw.to_csv(clean_csv_path, index=False)

print("Saved CSV to:", clean_csv_path)


Saved CSV to: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/clean/spy_1min_et_clean.csv
